In [49]:
# %pip install git+https://github.com/tensorflow/docs

import os.path

import numpy as np
import pandas as pd

from datetime import date
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [50]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH: date = date(2022, 7, 1)

TARGETNAME = 'placement'

In [51]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [52]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,UQsy8MuT5K-tevXeLzdDbpLaWEHv_W-fgHN_Rjvcyl1d7LS1,a0J5JOJeY2aVGdh6kI1nUfllToh0fsF-4fePC4bKBI8YPKfcKDhlEHXG,gIy-gJkaM-7b5m6zdVJ4W7u6_BfF4qXHo7mp-nXkSHT_6cd-1aKtr9fvDeDCziT0VLQaUZeNPmfFNw,ANO Vicer,4274,1657417780235,4,UQsy8MuT5K-tevXeLzdDbpLaWEHv_W-fgHN_Rjvcyl1d7LS1,ANO Vicer,791,I,165,131,False,False,False,False
1,w1r0HY8OZ6H8jWZo4ShHjQtfA7wa4-HU92DqNYTiIrldqlLr,eKn8to8_o299rFK63S9fhb7bEVlE1iaWOA4WV5XpjM-Aq7ZOWhCdpXRp,4NX7gAtZImPM8xv9ZtBJtFvO78BL0ZLqMHw1-L9lgkW6yVnu07StWdsLB8YurpUahdlWm16q1F4g0Q,GingGG,978,1657278563000,64,w1r0HY8OZ6H8jWZo4ShHjQtfA7wa4-HU92DqNYTiIrldqlLr,GingGG,554,I,64,36,False,False,True,False
2,2e1D6aQc1kpzeLJ2u_kN45lRvuy3bp1MjGBSOY1ezPnb2rw,CAc_fz6L09wqfEHoWjKhsj9jtySCqL4gUIWBQoKgvETbqw,v5KTPUAPKJSM4e_DqVMdqsTVttX5N6jv1m6_ICys0u1UpefvnJ4q7pGojOJMirZ8bbhgEvaoJ78xXg,UTO Qetzer,538,1657321952000,39,2e1D6aQc1kpzeLJ2u_kN45lRvuy3bp1MjGBSOY1ezPnb2rw,UTO Qetzer,614,I,152,118,False,False,True,False
3,D2GAOR3iab8h1iMD4NDcwb8Dlbf2P62kN53KfMkPtMBysUE,AWwdrw4jqPSSmGZnZswLd4ODp2LPwefw89TOewJt9JSbZQ,sGi2UMFcrkWy4FpKqTbicAhDTnPcNx2fe0tHQLWkaWnQVDwdSgSqZLkO846wO_UoyaqlpIUZGsb4yQ,Lylia,4540,1657438378000,101,D2GAOR3iab8h1iMD4NDcwb8Dlbf2P62kN53KfMkPtMBysUE,Lylia,548,I,92,61,False,False,True,False
4,RTYz-9FtUrVfHsVYl78M8HvZMqM1zxSnl5zqggYO549VdhjO,FHII_6hEp7SwKH0DrS1q0iDW6gKrONH8vCJCD0l58LuxtXoflHh0TODO,r3n06JtZGp4ns1fp8LwJBDvFHNzTk90DrDPC6S-zAP7x0O9S_WZrQhEMYqH4QVeC1dqKaLYYvv5zkw,DV1 Kacpersky191,28,1657411780000,9,RTYz-9FtUrVfHsVYl78M8HvZMqM1zxSnl5zqggYO549VdhjO,DV1 Kacpersky191,980,I,197,132,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,zf0zWfmkLZ6_PQVQRFQnGAEmChrtACqp8ftf80rP5mvCVFk,GbGcV03jVWQ1dhrpRA5aQfW58h5C6fWU9VHMszLYN8b1Klc,mhbLqKb-VJ_E-kbpe8P5wAoyJDrmOLSrxXXCHJIWovbjWyoCgpTIWiQVlzh5Bg6suzAkrXjnjfwkCQ,0slow,5090,1657408150000,100,zf0zWfmkLZ6_PQVQRFQnGAEmChrtACqp8ftf80rP5mvCVFk,0slow,582,I,196,174,False,False,False,False
196,NhOnvH1Y6gbKH502M9PtfWAF24gYb0HJjgjb-AsNvrbFug0,6Nu6-8JA0oTClfUU3G4pvTWh9MYspDiNixrpyIE41-VdDw,h0JSpbSTiLwT3wMLiS51AZGkTm-pIjL6uKeYjWkRVA9u_klsaZFLGuJYM1rEINnuOQT6htSaFTIPkQ,Adonis17,23,1657371317000,117,NhOnvH1Y6gbKH502M9PtfWAF24gYb0HJjgjb-AsNvrbFug0,Adonis17,688,I,139,103,False,False,True,False
197,ikgr2yw1rjgBf2VFtgln6rGtz4F8Hq4_eRiFEN7y3kwbG4TN,1_5uXe6sG5T8b1d7lHhqt4BzRbit8DIBha_NYZGt4FH-q58,kzTaWkjjaEGmOEb0pqzUlltiaCw4LfFG6_Lkbi3tRITep-T7TMQXfI1qml42S8-ounZXNggDXgs5qA,HeinoDoe,5292,1657393792000,117,ikgr2yw1rjgBf2VFtgln6rGtz4F8Hq4_eRiFEN7y3kwbG4TN,HeinoDoe,811,I,148,124,False,False,True,False
198,JBhOZjtikjeT-77UkFyVwlcpgX1FZNf0P50kAufEgs7MBLA,K4sNGULaA_d5qM7Mvobn02jsZ1_u-qPpS2cMwqp0pL82rw,bO-Rb8t-dHfAlTJ59Bt16vN_6jtBeJZV08TBeJxZVy_4T7f14ozif4M62vCAJp8jndb18udvEnU5SQ,BGN ShinoPavels,4275,1657252781000,86,JBhOZjtikjeT-77UkFyVwlcpgX1FZNf0P50kAufEgs7MBLA,BGN ShinoPavels,680,I,162,123,False,False,False,False


In [53]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [54]:
len(uniq_matches_id)

5915

In [55]:
len(matches_asset)

8460

In [56]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [57]:
len(uniq_matches)

5915

In [58]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [59]:
latest_matches = [match for match in uniq_matches if (
    LATEST_RELEASE in match['info']['game_version'])]


In [60]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [61]:
len(latest_matches)

5457

In [62]:
len(latest_patch_matches)

3677

In [63]:
# latest_matches[0]['info']

In [64]:
from typing import List


def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [65]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [66]:
matches_array[0]
# len(matches_array)

{'match_id': 'EUW1_5962513426',
 'placement': 6,
 'augment0': 'TFT6_Augment_Ascension',
 'augment1': 'TFT7_Augment_MirageTrait',
 'augment2': 'TFT7_Augment_CavalierEmblem',
 'Set7_Bruiser': 0,
 'Set7_Cavalier': 3,
 'Set7_Dragon': 1,
 'Set7_Dragonmancer': 0,
 'Set7_Evoker': 0,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Mirage': 1,
 'Set7_Mystic': 0,
 'Set7_Ragewing': 0,
 'Set7_Tempest': 2,
 'Set7_Trainer': 0,
 'Set7_Warrior': 0,
 'TFT7_Sejuani': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Sejuani_item0': 'SparringGloves',
 'TFT7_Nunu': 2,
 'TFT7_Nunu_item0': 'TitanicHydra',
 'TFT7_Lulu': 2,
 'TFT7_Lulu_item0': 'SpearOfShojin',
 'TFT7_Lulu_item1': 'SpearOfShojin',
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'RedBuff',
 'TFT7_Hecarim_item1': 'IonicSpark',
 'TFT7_Ornn': 1,
 'TFT7_Yasuo': 1,
 'TFT7_Yasuo_item0': 'CavalierEmblemItem',
 'TFT7_Yasuo_item1': 'MadredsBloodrazor',
 'TFT7_AoShin': 1,
 'TFT7_AoShin_item0': 'ArchangelsStaff',
 'TFT7_AoShin_ite

In [67]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [68]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,...,TFT7_Sett_item2,TFT7_Ezreal_item2,TFT7_Sejuani_item2,TFT7_Senna_item1,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Lillia_item2,TFT7_Senna_item2
0,EUW1_5962513426,6,TFT6_Augment_Ascension,TFT7_Augment_MirageTrait,TFT7_Augment_CavalierEmblem,0.0,3.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5962513426,3,TFT7_Augment_BruiserTitanicStrength,TFT6_Augment_TriForce1,TFT7_Augment_BigFriend2,3.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5962513426,8,TFT6_Augment_MetabolicAccelerator,TFT7_Augment_BandOfThieves1,TFT7_Augment_DragonHorde,0.0,NaN,2.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5962513426,7,TFT6_Augment_TrueTwos,TFT6_Augment_ThrillOfTheHunt1,TFT7_Augment_PandorasBench,NaN,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5962513426,4,TFT7_Augment_ShapeshifterBeastsDen,TFT6_Augment_Weakspot,TFT6_Augment_ThrillOfTheHunt2,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43651,EUW1_5956122676,3,TFT7_Augment_Bloodlust1,TFT7_Augment_BestFriends2,TFT6_Augment_SunfireBoard,2.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43652,EUW1_5956122676,2,TFT7_Augment_BestFriends2,TFT6_Augment_TradeSectorPlus,TFT6_Augment_JeweledLotus,2.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43653,EUW1_5956122676,6,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_ComponentGrabBag,TFT7_Augment_CannoneerRicochet,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43654,EUW1_5956122676,4,TFT6_Augment_HyperRoll,TFT6_Augment_ComponentGrabBag,TFT7_Augment_RevelPartyFavors,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,...,TFT7_Sett_item2,TFT7_Ezreal_item2,TFT7_Sejuani_item2,TFT7_Senna_item1,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Lillia_item2,TFT7_Senna_item2
0,EUW1_5962513426,6,TFT6_Augment_Ascension,TFT7_Augment_MirageTrait,TFT7_Augment_CavalierEmblem,0.0,3.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5962513426,3,TFT7_Augment_BruiserTitanicStrength,TFT6_Augment_TriForce1,TFT7_Augment_BigFriend2,3.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5962513426,8,TFT6_Augment_MetabolicAccelerator,TFT7_Augment_BandOfThieves1,TFT7_Augment_DragonHorde,0.0,NaN,2.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5962513426,7,TFT6_Augment_TrueTwos,TFT6_Augment_ThrillOfTheHunt1,TFT7_Augment_PandorasBench,NaN,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5962513426,4,TFT7_Augment_ShapeshifterBeastsDen,TFT6_Augment_Weakspot,TFT6_Augment_ThrillOfTheHunt2,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43651,EUW1_5956122676,3,TFT7_Augment_Bloodlust1,TFT7_Augment_BestFriends2,TFT6_Augment_SunfireBoard,2.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43652,EUW1_5956122676,2,TFT7_Augment_BestFriends2,TFT6_Augment_TradeSectorPlus,TFT6_Augment_JeweledLotus,2.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43653,EUW1_5956122676,6,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_ComponentGrabBag,TFT7_Augment_CannoneerRicochet,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43654,EUW1_5956122676,4,TFT6_Augment_HyperRoll,TFT6_Augment_ComponentGrabBag,TFT7_Augment_RevelPartyFavors,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [70]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [71]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

# Output dataframes

In [72]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)

# End